In [ ]:
# scFoundation

In [ ]:
# 1. Map RAW counts to samples

In [7]:
"""Map RAW counts to samples

Structure:
    1. Imports, Variables, Functions
    2. Load Data
"""

# 1. Imports, Variables, Functions
# imports
import numpy as np, pandas as pd, os, sys
import logging
from bs4 import BeautifulSoup

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# variables
html_content = "../data/DiSignAtlas/dsa_diff_download.individual_downloads/DSA10289"

# functions

# 2. Load Data

/tmp/ipykernel_8240/315372648.py:62: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(html_content, 'html.parser')


AttributeError: 'NoneType' object has no attribute 'find_all'

In [15]:
import requests
from bs4 import BeautifulSoup

# Define the URL
url = 'http://www.inbirg.com/disignatlas/detail/DSA00009'

# Send a GET request to fetch the HTML content
response = requests.get(url)
response.raise_for_status()  # Check if the request was successful

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')

# Function to extract "Control | Case" information
def extract_control_case_info(soup):
    info = {}

    # Find the table row with "Control | Case"
    table_rows = soup.find_all('tr')
    for row in table_rows:
        cells = row.find_all('td')
        if cells and 'Control | Case' in cells[0].get_text(strip=True):
            if len(cells) >= 3:
                control = cells[1].get_text(strip=True).split(';')
                case = cells[2].get_text(strip=True).split(';')
                info['Control'] = control
                info['Case'] = case
                break  # We found our row, so we can exit the loop

    return info

# Extract the "Control | Case" information
control_case_info = extract_control_case_info(soup)

# Print the extracted informationºº
print(control_case_info)


{'Control': ['GSM6943825', 'GSM6943826', 'GSM6943827', 'GSM6943828'], 'Case': ['GSM6943813', 'GSM6943814', 'GSM6943815', 'GSM6943816']}


In [2]:
import os
import requests
import pandas as pd
from bs4 import BeautifulSoup
from multiprocessing import Pool, cpu_count
from tqdm.contrib.concurrent import process_map

# Define function to extract "Control | Case" information
def extract_control_case_info(soup):
    info = {}

    # Find the table row with "Control | Case"
    table_rows = soup.find_all('tr')
    for row in table_rows:
        cells = row.find_all('td')
        if cells and 'Control | Case' in cells[0].get_text(strip=True):
            if len(cells) >= 3:
                control = cells[1].get_text(strip=True).split(';')
                case = cells[2].get_text(strip=True).split(';')
                info['Control'] = control
                info['Case'] = case
                break  # We found our row, so we can exit the loop

    return info

# Function to download file and extract information
def download_and_extract(dsaid):
    output_path = os.path.join("..", "data", "DiSignAtlas", "dsa_diff_download.individual_downloads/")
    download_url = f"http://www.inbirg.com/disignatlas/detail/{dsaid}"
    filename = f"{dsaid}.html"
    file_path = os.path.join(output_path, filename)

    max_retries = 10
    timeout = 10

    for attempt in range(max_retries):
        try:
            response = requests.get(download_url, timeout=timeout)
            if response.status_code == 200:
                with open(file_path, "wb") as file:
                    file.write(response.content)
                soup = BeautifulSoup(response.text, 'html.parser')
                info = extract_control_case_info(soup)
                return info
            else:
                pass
        except requests.RequestException as e:
            pass

    print(f"Failed to download {dsaid} after {max_retries} attempts.")
    return None


# Function to process dsaids in parallel
def process_dsaids(dsaid_list):
    with Pool(processes=8) as pool:  # Adjust the number of processes as needed
        results = pool.map(download_and_extract, dsaid_list)
    return results


# Load Identifiers
data_path = os.path.join(
    "..", "data", "DiSignAtlas", "Disease_information_Datasets.csv"
)
df = pd.read_csv(data_path)
dsaids = df["dsaid"].tolist()


In [3]:
cpu_count()

16

In [2]:
extracted_info = process_map(download_and_extract, dsaids, max_workers=16, chunksize=32)

  0%|          | 0/10306 [00:00<?, ?it/s]

Failed to download DSA02705 after 10 attempts.


In [ ]:

# Save results to a CSV file
extracted_df = pd.DataFrame(extracted_info)
extracted_df.to_csv('extracted_control_case_info.csv', index=False)
print("Extraction and download complete.")


In [ ]:
external_links_data_path = os.path.join(
    "..", "data", "DiSignAtlas", "external_links.pkl"
)

import pickle

a = pickle.load(open(external_links_data_path, "rb"))

In [ ]:
# 2. Map to Gene Symbols

In [ ]:
# 3. Store in Matrix